# Mystery box - 5 - Find the goal
### Dennis Bakhuis - 10th November 2022
### https://linkedin.com/in/dennisbakhuis/

In [1]:
import folium
from folium.plugins import HeatMap
import pandas as pd
from tqdm.auto import tqdm

In [2]:
gps = pd.read_parquet('../data/mystery_box_dataset.parquet')

## Lets create a map, just for fun

In [3]:
mymap = folium.Map(
    location=[52.219433827394745, 6.896132071889914], 
    zoom_start=14, 
    tiles="Stamen Watercolor",
    
)

data = gps[['latitude', 'longitude', 'A']].values.tolist()

HeatMap(data, name='Heart rate', radius=20).add_to(mymap)
mymap

## Create an error function and minimize

In [4]:
import haversine
from scipy.optimize import minimize
import pandas as pd

In [5]:
def func(a, b, c, d, x1, y1, A):
    d1 = haversine.haversine((a, b), (x1, y1), unit='m')
    d2 = haversine.haversine((c, d), (x1, y1), unit='m')
    return abs(d1 - d2 - A)

def calc_error(X):
    a, b, c, d = X[0], X[1], X[2], X[3]
    error = 0
    for row in gps.itertuples():
        A = row.A
        x1 = row.latitude
        y1 = row.longitude
        error += func(a, b, c, d, x1, y1, A)
    return error / len(gps)


bounds = [
    (45, 60), (-10, 16),
    (45, 60), (-10, 16),
]

# Some random guesses in Enschede  
X0 = [  
    52.214440, 6.881985,
    52.223679, 6.907095,
]

Minimize to find p1 and p2.

In [11]:
res = minimize(
    calc_error,
    x0=X0,
    bounds=bounds,
    tol=1e-4,
)

x = res.x
p1, p2 = (x[0], x[1]), (x[2], x[3])
goal = (x[0] + x[2]) /2, (x[1] + x[3]) / 2
print(f"Goal location: {goal}") 

Goal location: (52.2658475946842, 6.876935563709133)


## Plot it on the map

In [8]:
goal

(52.2658475946842, 6.876935563709133)

In [9]:
zero_points = (
    gps
    .loc[gps.distance.abs() <= 1, ['latitude', 'longitude']]
    .to_records(index=False)
    .tolist()
)

In [10]:
mymap = folium.Map(
    location=[52.24924502556542, 6.876265833857593], 
    zoom_start=12,    
)

# Add locations
folium.Marker(location=goal, icon=folium.Icon(color='green', icon='flag')).add_to(mymap)
folium.Marker(location=p1, icon=folium.Icon(color='blue', icon='flag')).add_to(mymap)
folium.Marker(location=p2, icon=folium.Icon(color='blue', icon='flag')).add_to(mymap)

for zero_point in zero_points:
    folium.Marker(location=zero_point, icon=folium.Icon(color='red', icon='bell')).add_to(mymap)

folium.PolyLine([goal, zero_points[1]]).add_to(mymap)
folium.PolyLine([p1, p2]).add_to(mymap)
    
mymap

## Detailed map

In [26]:
marcook = (52.26503679705878, 6.876591022191117)
map_center = ((marcook[0]+goal[0])/2, (marcook[1]+goal[1])/2)

distance = haversine.haversine(goal, marcook, unit=haversine.Unit.METERS)
print(f"Distance: {distance:.2f} meters")

Distance: 93.16 meters


In [20]:
mymap = folium.Map(
    location=map_center, 
    zoom_start=18,    
)

# Add locations
folium.Marker(location=goal, icon=folium.Icon(color='green', icon='flag')).add_to(mymap)
folium.Marker(location=marcook, icon=folium.Icon(color='red', icon='flag')).add_to(mymap)
folium.PolyLine([goal, marcook]).add_to(mymap)

mymap

93.15567078067681